In [11]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [12]:
url_construct = "https://fme.discomap.eea.europa.eu/fmedatastreaming/AirQualityDownload/AQData_Extract.fmw?CountryCode={}&CityName={}&Pollutant={}&Year_from=2016&Year_to=2020&Station=&Samplingpoint=&Source=E1a&Output=HTML&UpdateDate=&TimeCoverage=Year"

In [13]:
cities = [{'country': 'GB', 'city': 'London', 'weather': 'london'},
          {'country': 'GB', 'city': 'Greater%20Manchester', 'weather': 'Manchester'},
          {'country': 'NL', 'city': 'Amsterdam','weather': 'Amsterdam'},
          {'country': 'IT', 'city': 'Milano','weather': 'Milano'},
          {'country': 'FR', 'city': 'Paris','weather': 'Paris'},
          {'country': 'ES', 'city': 'Madrid','weather':'Madrid'},
          {'country': 'DE', 'city': 'Berlin', 'weather':'Berlin'},
          {'country': 'DE', 'city': 'Frankfurt%20am%20Main','weather': 'Frankfurt'},
         ]
pollutants = ['5', '6001', '9', '8']

In [16]:
# # Test stuff
# soup = BeautifulSoup(r.content, 'lxml')
# links = []
# for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
#     links.append(link.get('href'))

In [40]:
# master_df = pd.DataFrame()

In [4]:
# url = "https://fme.discomap.eea.europa.eu/fmedatastreaming/AirQualityDownload/AQData_Extract.fmw?CountryCode=GB&CityName=London&Pollutant=5Year_from=2016&Year_to=2020&Station=&Samplingpoint=&Source=E1a&Output=HTML&UpdateDate=&TimeCoverage=Year"
# r = requests.get(url)
# print(r.content)
# soup = BeautifulSoup(r.content, 'lxml')
# links = []
# for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
#     links.append(link.get('href'))
# df = pd.read_csv(links[0])
# df['datetime'] = pd.to_datetime(df['DatetimeBegin'], errors='coerce', infer_datetime_format=True) 
# df['hour'] = df['datetime'].dt.hour
# df['year'] = df['datetime'].dt.year
# df['day'] = df['datetime'].dt.day
# df

b''


IndexError: list index out of range

In [ ]:
low = df[df['hour'] == 2]
high = df[df['hour'] == 17]
dat = {
    'city': 'London',
    'site': df['AirQualityStation'].iloc[0],
    'pollutant': df['AirPollutant'].iloc[0],
    'year': df['year'].iloc[0],
    'low': low['Concentration'].mean(),
    'high': high['Concentration'].mean(),
    'measurement': df['UnitOfMeasurement'].iloc[0]
}
master_df = master_df.append(dat, ignore_index=True)

In [15]:
master_df = pd.DataFrame()
all_data = pd.DataFrame()
daily_df = pd.DataFrame()
for city in cities:
    for pollutant in pollutants:
        url = url_construct.format(city['country'], city['city'], pollutant)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'lxml')
        links = []
        daily_city_pol = pd.DataFrame()
        for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
            links.append(link.get('href'))
        for link in links:
            df = pd.read_csv(link)
            df['datetime'] = pd.to_datetime(df['DatetimeBegin'], errors='coerce', infer_datetime_format=True) 
            df['hour'] = df['datetime'].dt.hour
            df['year'] = df['datetime'].dt.year
            df['date'] = df['datetime'].dt.date
            low = df[df['hour'] == 2]
            high = df[df['hour'] == 17]
            midnight = df[df['hour'] == 0]
            dat = {
                'city': city['city'],
                'site': df['AirQualityStation'].iloc[0],
                'pollutant': df['AirPollutant'].iloc[0],
                'year': df['year'].iloc[0],
                'low': low['Concentration'].mean(),
                'high': high['Concentration'].mean(),
                'midnight': midnight['Concentration'].mean(),
                'measurement': df['UnitOfMeasurement'].iloc[0]
            }
            master_df = master_df.append(dat, ignore_index=True)
            daily_site_poll = df.groupby('date', as_index=False).mean()
            low['2am'] = low['Concentration'].copy()
            high['5pm'] = high['Concentration'].copy()
            midnight['0am'] = midnight['Concentration'].copy()
            daily_site_poll = pd.merge(daily_site_poll, low[['2am', 'date']], left_on='date', right_on='date', how='left')
            daily_site_poll = pd.merge(daily_site_poll, high[['5pm', 'date']], on='date', how='left')
            daily_site_poll = pd.merge(daily_site_poll, midnight[['0am', 'date']], on='date', how='left')
            weather_df = pd.read_csv('outputs/' + city['weather'] + '_weather.csv')
            weather_df['datetime'] = pd.to_datetime(weather_df['Date time'], errors='coerce', infer_datetime_format=True)
            weather_df['date'] = weather_df['datetime'].dt.date
            daily_site_poll = pd.merge(daily_site_poll, weather_df[['Temperature', 'Wind Speed', 'Precipitation', 'date']], left_on='date', right_on='date', how='left')
            daily_site_poll['city'] = city['city']
            daily_site_poll['measurement'] = df['UnitOfMeasurement'].iloc[0]
            daily_site_poll['site'] = df['AirQualityStation'].iloc[0]
            daily_site_poll['pollutant'] = df['AirPollutant'].iloc[0]
            daily_city_pol = daily_city_pol.append(daily_site_poll)
            daily_site_poll.to_csv('outputs/daily_site/' + city['city'] + '_' + df['AirPollutant'].iloc[0] + '_' + str(df['AirQualityStation'].iloc[0]) + '.csv', index=False)
        print(daily_city_pol.head())
        try:
            daily_city_pol = daily_city_pol.groupby(['city', 'measurement', 'pollutant', 'date'], as_index=False).mean()
        except KeyError:
            pass
        daily_city_pol.to_csv('outputs/daily_city/' + city['city'] + '_' + df['AirPollutant'].iloc[0] + '.csv', index=False)
        all_data = all_data.append(daily_city_pol)
        


C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarn

         date  AirQualityStationEoICode  Concentration  Validity  \
0  2016-01-01                       NaN      22.920833       1.0   
1  2016-01-02                       NaN      11.691667       1.0   
2  2016-01-03                       NaN      12.887500       1.0   
3  2016-01-04                       NaN      17.404167       1.0   
4  2016-01-05                       NaN      17.366667       1.0   

   Verification  hour    year   2am   5pm   0am  Temperature  Wind Speed  \
0           1.0  11.5  2016.0  51.0  18.9  17.6          6.5         0.2   
1           1.0  11.5  2016.0   8.2   7.9  11.7         10.4         0.2   
2           1.0  11.5  2016.0   6.2  11.3  11.1          8.8         0.2   
3           1.0  11.5  2016.0  23.4  22.0  25.6          9.2         0.1   
4           1.0  11.5  2016.0   9.9  21.7  16.9          9.0         0.1   

   Precipitation    city measurement             site pollutant  
0            0.0  London       µg/m3  Station_GB0620A      PM10  
1 

C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarn

         date  AirQualityStationEoICode  Concentration  Validity  \
0  2016-01-01                       NaN       6.991667  1.250000   
1  2016-01-02                       NaN       2.500000  1.958333   
2  2016-01-03                       NaN       2.395000  1.250000   
3  2016-01-04                       NaN       3.862500  1.666667   
4  2016-01-05                       NaN       4.254167  1.583333   

   Verification  hour    year  2am  5pm  0am  Temperature  Wind Speed  \
0           1.0  11.5  2016.0  8.6  7.8  6.1          6.5         0.2   
1           1.0  11.5  2016.0  0.3  4.2 -0.7         10.4         0.2   
2           1.0  11.5  2016.0 -0.1 -0.3  4.8          8.8         0.2   
3           1.0  11.5  2016.0  5.8  5.8  7.3          9.2         0.1   
4           1.0  11.5  2016.0  3.0  7.8  4.4          9.0         0.1   

   Precipitation    city measurement             site pollutant  
0            0.0  London       µg/m3  Station_GB0566A     PM2.5  
1            0.8  Lo

C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarn

...

         date  AirQualityStationEoICode  Concentration  Validity  \
0  2016-01-01                       NaN      39.582804       1.0   
1  2016-01-02                       NaN       9.790495       1.0   
2  2016-01-03                       NaN      11.543624       1.0   
3  2016-01-04                       NaN      12.088699       1.0   
4  2016-01-05                       NaN       6.789407       1.0   

   Verification  hour    year  2am  5pm        0am  Temperature  Wind Speed  \
0           1.0   0.0  2016.0  NaN  NaN  39.582804          5.2         8.1   
1           1.0   0.0  2016.0  NaN  NaN   9.790495          1.7        14.8   
2           1.0   0.0  2016.0  NaN  NaN  11.543624          3.3        13.8   
3           1.0   0.0  2016.0  NaN  NaN  12.088699          2.9         8.1   
4           1.0   0.0  2016.0  NaN  NaN   6.789407          5.0        10.5   

   Precipitation                   city measurement            site pollutant  
0            0.0  Frankfurt%20am%20M

C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarn

         date  AirQualityStationEoICode  Concentration  Validity  \
0  2016-01-01                       NaN      52.025345       1.0   
1  2016-01-02                       NaN      65.072655       1.0   
2  2016-01-03                       NaN      61.546536       1.0   
3  2016-01-04                       NaN     119.128543       1.0   
4  2016-01-05                       NaN      69.666763       1.0   

   Verification  hour    year        2am         5pm        0am  Temperature  \
0           1.0  11.5  2016.0  53.891548   60.856186  86.185776          5.2   
1           1.0  11.5  2016.0  21.604965  125.779480  35.573700          1.7   
2           1.0  11.5  2016.0  40.049309   59.898388  68.917526          3.3   
3           1.0  11.5  2016.0  30.701267  216.361496  38.542759          2.9   
4           1.0  11.5  2016.0  26.004251  115.512177  37.941406          5.0   

   Wind Speed  Precipitation                   city measurement  \
0         8.1            0.0  Frankfurt%20a

C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\russell.plunkett\AppData\Local\Continuum\anaconda37\envs\ucl_analytics_for_policy\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarn

         date  AirQualityStationEoICode  Concentration  Validity  \
0  2016-01-01                       NaN      34.701168       1.0   
1  2016-01-02                       NaN      33.335256       1.0   
2  2016-01-03                       NaN      33.070471       1.0   
3  2016-01-04                       NaN      48.320827       1.0   
4  2016-01-05                       NaN      45.403513       1.0   

   Verification  hour    year        2am        5pm        0am  Temperature  \
0           1.0  11.5  2016.0  39.627205  40.185177  42.350319          5.2   
1           1.0  11.5  2016.0  17.490273  49.534187  26.455925          1.7   
2           1.0  11.5  2016.0  30.070671  34.208439  37.643036          3.3   
3           1.0  11.5  2016.0  22.566221  70.562576  26.026886          2.9   
4           1.0  11.5  2016.0  25.185390  68.025978  31.065470          5.0   

   Wind Speed  Precipitation                   city measurement  \
0         8.1            0.0  Frankfurt%20am%20Ma

In [20]:
all_data.replace('Frankfurt%20am%20Main', 'Frankfurt', inplace=True)
all_data.replace('Greater%20Manchester', 'Manchester', inplace=True)


In [21]:
all_data.to_csv('outputs/daily_all_cities_all_pollutants.csv', index=False)

In [17]:
master_df.to_csv('outputs/pollution_city_comparison.csv', index=False)